In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.serialization
from pathlib import Path

from torch_geometric.data import HeteroData
from torch_geometric.nn import HGTConv, global_mean_pool
from torch_geometric.loader import DataLoader
import analysis_utils

# Allow loading HeteroData under PyTorch 2.6+
torch.serialization.add_safe_globals([HeteroData])

############################################################
# 1. LOAD GRAPHS
############################################################

def load_graphs(graph_dir):
    graph_dir = Path(graph_dir)
    graphs = []
    for f in sorted(graph_dir.glob("graph_*.pt")):
        g = torch.load(f, weights_only=False)
        graphs.append(g)
    return graphs


############################################################
# 2. MODEL: SpeechHeteroGNN (HGTConv + edge aggregation)
############################################################

class SpeechHeteroGNN(nn.Module):
    """
    Graph neural network that:
    - Embeds author/topic/day via ID embeddings
    - Appends aggregated edge attributes (score, lag, decay)
      to speech node features
    - Runs an HGTConv message passing layer
    """

    def __init__(self, global_idx, hidden_dim=64, num_heads=2):
        super().__init__()
        self.hidden_dim = hidden_dim

        # Number of items for embedding lookup
        num_authors = len(global_idx["author2idx"]) + 1
        num_topics  = len(global_idx["topic2idx"]) + 1
        num_days    = len(global_idx["date2idx"]) + 1

        # Embeddings for ID-based nodes
        self.emb = nn.ModuleDict({
            "author": nn.Embedding(num_authors, hidden_dim),
            "topic":  nn.Embedding(num_topics, hidden_dim),
            "day":    nn.Embedding(num_days, hidden_dim),
        })

        # Speech projection:
        #   768-dimensional embedding
        # + 3 extra features (mean score, lag, decay)
        # = 771 → hidden_dim
        self.speech_lin = nn.Linear(768 + 3, hidden_dim)

        # HGTConv (heterogeneous transformer layer)
        self.hgt = HGTConv(
            in_channels=hidden_dim,
            out_channels=hidden_dim,
            metadata=(
                ["author", "speech", "topic", "day"],
                [
                    ("author", "gives", "speech"),
                    ("speech", "rev_gives", "author"),
                    ("speech", "mentions", "topic"),
                    ("topic", "rev_mentions", "speech"),
                    ("day", "references", "speech"),
                    ("speech", "rev_references", "day"),
                ]
            ),
            heads=num_heads,
        )


    ############################################################
    # Aggregation of edge attributes → speech node features
    ############################################################
    def compute_edge_features(self, data, device):
        """
        Returns [num_speech, 3]:
        [mean_topic_score, mean_lag, mean_decay]
        """

        Ns = data["speech"].x.size(0)
        extras = torch.zeros(Ns, 3, device=device)
        count  = torch.zeros(Ns, 3, device=device)

        # 1) Topic score: speech → topic edges
        if ("speech","mentions","topic") in data.edge_types:
            store = data["speech","mentions","topic"]
            if hasattr(store, "edge_attr") and store.edge_attr is not None:
                src   = store.edge_index[0]              # speech index
                score = store.edge_attr.view(-1).to(device)
                extras[:,0].index_add_(0, src, score)
                count[:,0].index_add_(0, src, torch.ones_like(score))

        # 2) Lag + decay: day → speech edges
        if ("day","references","speech") in data.edge_types:
            store = data["day","references","speech"]
            if hasattr(store, "edge_attr") and store.edge_attr is not None:
                dst    = store.edge_index[1]
                lagdec = store.edge_attr.to(device)     # [E,2]
                extras[:,1:].index_add_(0, dst, lagdec)
                count[:,1:].index_add_(0, dst, torch.ones_like(lagdec))

        count = torch.clamp(count, min=1)
        return extras / count  # shape [Ns, 3]


    ############################################################
    # Forward Pass
    ############################################################
    def forward(self, data: HeteroData):
        device = data["speech"].x.device
        x_dict = {}

        # ------- ID Nodes (author, topic, day) -------
        for ntype in ["author","topic","day"]:
            idx = data[ntype].x.long().view(-1)
            x_dict[ntype] = self.emb[ntype](idx)   # [N, hidden_dim]

        # ------- Speech Nodes -------
        base_feats = data["speech"].x.float().to(device)     # [N, 768]
        edge_feats = self.compute_edge_features(data, device)  # [N, 3]

        speech_full = torch.cat([base_feats, edge_feats], dim=-1)
        x_dict["speech"] = self.speech_lin(speech_full)

        # ------- HGTConv message passing -------
        x_dict = self.hgt(x_dict, data.edge_index_dict)

        return x_dict



############################################################
# 3. TOP MODEL: Predict from day-node
############################################################

class FedSpeechModel(nn.Module):
    def __init__(self, global_idx, hidden_dim=64):
        super().__init__()
        self.gnn = SpeechHeteroGNN(global_idx, hidden_dim)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, g: HeteroData):
        x = self.gnn(g)
        day_emb = x["day"]          # [1, hidden_dim]
        return self.fc(day_emb).squeeze()   # scalar prediction



############################################################
# 4. TRAINING AND EVALUATION
############################################################

def train_epoch(model, loader, optimizer, device):
    model.train()
    total_loss = 0
    n = 0

    for g in loader:
        g = g.to(device)
        target = g.y.to(device).float()

        pred = model(g)
        loss = F.mse_loss(pred, target)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        total_loss += loss.item()
        n += 1

    return total_loss / n


@torch.no_grad()
def eval_epoch(model, loader, device):
    model.eval()
    total_loss = 0
    n = 0

    for g in loader:
        g = g.to(device)
        target = g.y.to(device).float()

        pred = model(g)
        loss = F.mse_loss(pred, target)

        total_loss += loss.item()
        n += 1

    return total_loss / n



############################################################
# 5. TRAIN MODEL
############################################################

def train_model(graphs, global_idx, hidden_dim=64, epochs=50):
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Remove graphs without speech nodes
    graphs = [g for g in graphs if g["speech"].x.size(0) > 0]

    # Chronological split
    n = len(graphs)
    n_train = int(n * 0.8)
    train_graphs = graphs[:n_train]
    val_graphs   = graphs[n_train:]

    train_loader = DataLoader(train_graphs, batch_size=1, shuffle=True)
    val_loader   = DataLoader(val_graphs, batch_size=1, shuffle=False)

    model = FedSpeechModel(global_idx, hidden_dim).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)

    print(f"Training on device: {device}")
    print(f"Graphs: {n}, Train: {len(train_graphs)}, Val: {len(val_graphs)}")

    best_val = float("inf")
    best_state = None
    patience = 20
    bad_epochs = 0

    for epoch in range(1, epochs+1):
        train_loss = train_epoch(model, train_loader, optimizer, device)
        val_loss   = eval_epoch(model, val_loader, device)

        print(f"Epoch {epoch:03d} | Train={train_loss:.4f} | Val={val_loss:.4f}")

        if val_loss < best_val - 1e-6:
            best_val = val_loss
            best_state = model.state_dict()
            bad_epochs = 0
        else:
            bad_epochs += 1

        if bad_epochs >= patience:
            print("Early stopping.")
            break

    model.load_state_dict(best_state)
    print("Best Val MSE:", best_val)
    return model


# Build global_idx the same way you did before building graphs
def build_global_indices(speeches, topic_scores, rates_df):
    author_names = sorted({v["author"] for v in speeches.values()})
    author2idx = {name: i for i, name in enumerate(author_names)}

    topic_names = sorted({t for sid in topic_scores for t in topic_scores[sid].keys()})
    topic2idx = {t: i for i, t in enumerate(topic_names)}

    date_names = sorted(rates_df.index)
    date2idx = {d: i for i, d in enumerate(date_names)}

    return {
        "author2idx": author2idx,
        "topic2idx": topic2idx,
        "date2idx": date2idx,
    }

############################################################
# 6. SCRIPT ENTRYPOINT
############################################################

if __name__ == "__main__":
    graphs = load_graphs("graphs_ffr_delta")
    
    speeches = analysis_utils.load_speeches()
    topic_scores = analysis_utils.load_topic_scores_by_sid()
    rates_df = analysis_utils.load_rates()


    # You should load global_idx from your preprocessing
    # Here is a placeholder example:
    #global_idx = torch.load("global_idx.pt")  # or construct manually
    global_idx = build_global_indices(speeches, topic_scores, rates_df)

    model = train_model(graphs, global_idx, hidden_dim=64, epochs=100)


C:\Users\MainUser\project\cs224w_cb_graph\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Training on device: cpu
Graphs: 647, Train: 517, Val: 130


RuntimeError: mat1 and mat2 shapes cannot be multiplied (12x772 and 771x64)